In [1]:
%load_ext autoreload
%autoreload 2

<h1>Оглавление<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#Data-loader" data-toc-modified-id="Data-loader-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Data loader</a></span></li></ul></div>

In [2]:
from data_handlers.data_preparer import get_datasets
import torch
from typing import Dict
from viewer import Viewer

In [3]:
import seaborn as sns
import pandas as pd

# Train loop

In [4]:
from itertools import product

In [5]:
from data_handlers.data_preparer import get_datasets

In [6]:
datasets = get_datasets()

Found annotations at '/mnt/heap/raw/instances_val2017.json'
Images already downloaded
Existing download of split 'validation' is sufficient
Loading 'coco-2017' split 'validation'
 100% |███████████████| 5000/5000 [21.6s elapsed, 0s remaining, 249.8 samples/s]      
Dataset 'coco-2017-validation' created
person_view len = 1005
train dataset = 753
test dataset = 252


In [7]:
def get_loaders(datasets) -> Dict[str,  torch.utils.data.DataLoader]:
    return {
        'train': torch.utils.data.DataLoader(
            datasets['train'], batch_size=64, shuffle=True),
        'val': torch.utils.data.DataLoader(
            datasets['val'], batch_size=64, shuffle=False)
    }

In [8]:
loaders = get_loaders(datasets)

In [9]:
viewer = Viewer()

In [10]:
datasets['train'][0]

(tensor([[[0.7922, 0.8000, 0.8000,  ..., 0.7529, 0.7490, 0.7412],
          [0.7686, 0.7882, 0.8039,  ..., 0.7412, 0.7490, 0.7412],
          [0.7804, 0.7765, 0.7882,  ..., 0.7373, 0.7490, 0.7333],
          ...,
          [0.7725, 0.7569, 0.7608,  ..., 0.8196, 0.8275, 0.8353],
          [0.7647, 0.7647, 0.7569,  ..., 0.8235, 0.8235, 0.8314],
          [0.7529, 0.7725, 0.7686,  ..., 0.8353, 0.8353, 0.8353]],
 
         [[0.7765, 0.7843, 0.7804,  ..., 0.7333, 0.7294, 0.7216],
          [0.7529, 0.7686, 0.7804,  ..., 0.7255, 0.7373, 0.7255],
          [0.7608, 0.7569, 0.7686,  ..., 0.7255, 0.7373, 0.7216],
          ...,
          [0.7569, 0.7569, 0.7608,  ..., 0.8118, 0.8196, 0.8275],
          [0.7569, 0.7608, 0.7529,  ..., 0.8157, 0.8118, 0.8157],
          [0.7490, 0.7686, 0.7647,  ..., 0.8275, 0.8235, 0.8196]],
 
         [[0.7765, 0.7882, 0.7882,  ..., 0.7608, 0.7529, 0.7451],
          [0.7647, 0.7804, 0.7882,  ..., 0.7529, 0.7608, 0.7529],
          [0.7804, 0.7725, 0.7765,  ...,